In [1]:
import tensorflow as tf
tf.__version__
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8133552907824534086,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 15581516555459346216
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6918604064
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 15040998855063717675
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8199571755367449316
 physical_device_desc: "device: XLA_GPU device"]

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
tf.debugging.set_log_device_placement(True)

a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)
print(c)

Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [4]:
import keras
from keras.models import Sequential, Model
from keras import layers, applications, optimizers
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D
from keras.preprocessing.image import ImageDataGenerator, load_img

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import random

In [5]:
epoch = 100
batch_size = 5
image_size = 224

In [6]:
#load train data(25000 imgs each)

filenames = os.listdir('./input/train/train')
data = []
categories = []

for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)
    
    image = keras.preprocessing.image.load_img(os.path.join('./input/train/train', filename),
                                               color_mode='rgb',
                                               target_size= (image_size,image_size))
    image = np.array(image)
    data.append(image)

data = np.array(data)
categories = np.array(categories)

print(data)
print(categories)

[[[[203 164  87]
   [206 167  90]
   [209 170  93]
   ...
   [245 203 119]
   [241 202 123]
   [239 200 121]]

  [[203 164  87]
   [206 167  90]
   [209 170  93]
   ...
   [245 205 120]
   [242 203 124]
   [240 201 122]]

  [[203 164  87]
   [206 167  90]
   [209 170  93]
   ...
   [245 204 122]
   [243 204 125]
   [241 202 123]]

  ...

  [[154 123  56]
   [155 124  57]
   [156 125  58]
   ...
   [  3   3   1]
   [  3   3   1]
   [  3   3   1]]

  [[153 122  55]
   [153 122  55]
   [154 123  56]
   ...
   [  2   2   0]
   [  2   2   0]
   [  2   2   0]]

  [[151 120  53]
   [152 121  54]
   [153 122  55]
   ...
   [  1   1   0]
   [  1   1   0]
   [  1   1   0]]]


 [[[ 39  44  40]
   [ 40  44  43]
   [ 41  45  46]
   ...
   [210 209 181]
   [207 204 171]
   [201 199 161]]

  [[ 40  45  41]
   [ 40  44  43]
   [ 41  45  46]
   ...
   [207 203 176]
   [203 200 169]
   [197 195 157]]

  [[ 39  44  40]
   [ 38  42  41]
   [ 37  41  42]
   ...
   [195 191 166]
   [198 193 164]
   [205 200

In [7]:
# shuffle dataset

idx = np.random.permutation(len(data))
data, categories = data[idx], categories[idx]

In [9]:
# split train 6(15,000) / val 2(5,000) / test 2(5,000)

x_train = data[:15000]
y_train = categories[:15000]
x_temp, y_temp = data[-10000:], categories[-10000:]

from sklearn.model_selection import train_test_split

x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5)

In [10]:
print(x_train, y_train)

print(x_temp, y_temp)
print(x_temp.shape, y_temp.shape)

print(x_val, y_val)

print(x_test, y_test)
print(x_test.shape, y_test.shape)

[[[[192 191 197]
   [210 209 215]
   [185 184 190]
   ...
   [195 192 199]
   [198 195 202]
   [179 178 186]]

  [[199 198 204]
   [205 204 210]
   [194 193 199]
   ...
   [195 192 199]
   [181 178 185]
   [182 181 189]]

  [[197 196 202]
   [196 195 201]
   [193 192 198]
   ...
   [197 194 201]
   [179 176 183]
   [195 194 202]]

  ...

  [[253 253 253]
   [248 248 248]
   [251 251 251]
   ...
   [248 246 247]
   [244 242 243]
   [245 245 245]]

  [[251 251 251]
   [248 248 248]
   [251 251 251]
   ...
   [247 245 246]
   [245 243 244]
   [242 242 242]]

  [[247 247 247]
   [250 250 250]
   [250 250 250]
   ...
   [240 238 239]
   [243 241 242]
   [248 248 248]]]


 [[[141 146 140]
   [142 147 141]
   [144 149 143]
   ...
   [186 192 182]
   [188 194 184]
   [180 186 176]]

  [[142 147 141]
   [143 148 142]
   [144 149 143]
   ...
   [179 185 175]
   [174 180 170]
   [175 181 171]]

  [[142 147 141]
   [143 148 142]
   [145 150 144]
   ...
   [180 186 176]
   [186 192 182]
   [180 186

In [11]:
def build_vgg_block(input_layer, num_cnn=3, channel=64, block_num=1):
    x = input_layer
    
    for cnn_num in range(num_cnn):
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(3,3),
            activation='relu',
            kernel_initializer='he_normal',
            padding='same',
            name=f'block{block_num}_conv{cnn_num}')(x)

    x = keras.layers.MaxPooling2D(
        pool_size=(2, 2),
        strides=2,
        name=f'block{block_num}_pooling')(x)

    return x

In [12]:
def build_vgg(input_shape=(image_size,image_size,3),
              num_cnn_list=[2,2,3,3,3],
              channel_list=[64,128,256,512,512],
              num_classes=2):
    input_layer = keras.layers.Input(shape=input_shape)
    output = input_layer
    
    for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
        output = build_vgg_block(
            output,
            num_cnn=num_cnn, 
            channel=channel,
            block_num=i
    )
        
    output = keras.layers.Flatten(name='flatten')(output)
    output = keras.layers.Dense(4096, activation='relu', kernel_regularizer='l2', name='fc1')(output)
    output = keras.layers.Dropout(0.5)(output)
    output = keras.layers.Dense(4096, activation='relu', kernel_regularizer='l2', name='fc2')(output)
    output = keras.layers.Dropout(0.5)(output)
    output = keras.layers.Dense(num_classes, activation='softmax', name='predictions')(output)
    
    model = keras.Model(
        inputs=input_layer, 
        outputs=output
    )
    return model

In [13]:
%load_ext tensorboard

import datetime

In [14]:
model = build_vgg()

Executing op TruncatedNormal in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/re

Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op TruncatedNormal in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localh

Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/

In [15]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, clipnorm=1.),
    metrics=['accuracy']
)

Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:lo

In [16]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block0_conv0 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block0_conv1 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block0_pooling (MaxPooling2D (None, 112, 112, 64)      0         
_________________________________________________________________
block1_conv0 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block1_pooling (MaxPooling2D (None, 56, 56, 128)      

In [17]:
log_dir = "logs/vgg_minibatch/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [18]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
model_checkpoint = ModelCheckpoint('vgg16.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

In [19]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.00001)

In [ ]:
# without early stopping

epoch = 50

history = model.fit(
    x_train, y_train,
    epochs=epoch,
    validation_data=(x_val, y_val),
    validation_steps=x_val.size//batch_size,
    steps_per_epoch=x_test.size//batch_size,
    verbose=1,
    #callbacks=[tensorboard_callback, early_stopping, model_checkpoint]
    callbacks=[tensorboard_callback, reduce_lr]
)

Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op SummaryWriter 

Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device 

Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:loc

       24/150528000 [..............................] - ETA: 2083:26:48 - loss: 280.6432 - accuracy: 0.5833Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
       26/150528000 [..............................] - ETA: 2083:36:59 - loss: 267.3820 - accuracy: 0.6154Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
       28/150528000 [..............................] - ETA: 2083:40:52 - loss: 257.6363 - accuracy: 0.6071Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
       31/150528000 [..............................] - ETA: 2083:51:34 - loss: 245

       75/150528000 [..............................] - ETA: 2087:37:20 - loss: 182.0719 - accuracy: 0.6000Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
       78/150528000 [..............................] - ETA: 2088:01:46 - loss: 179.0006 - accuracy: 0.5897Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
       80/150528000 [..............................] - ETA: 2087:54:02 - loss: 177.0232 - accuracy: 0.6000Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
       85/150528000 [..............................] - ETA: 2089:39:20 - loss: 172

...

     3139/150528000 [..............................] - ETA: 2087:32:15 - loss: 20.1654 - accuracy: 0.4967Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
     3141/150528000 [..............................] - ETA: 2087:31:41 - loss: 20.1526 - accuracy: 0.4970Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
     3143/150528000 [..............................] - ETA: 2087:30:49 - loss: 20.1400 - accuracy: 0.4973Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
     3146/150528000 [..............................] - ETA: 2087:30:48 - loss: 20.122

     3189/150528000 [..............................] - ETA: 2087:22:03 - loss: 19.8676 - accuracy: 0.4964Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
     3191/150528000 [..............................] - ETA: 2087:21:13 - loss: 19.8553 - accuracy: 0.4967Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
     3193/150528000 [..............................] - ETA: 2087:20:21 - loss: 19.8442 - accuracy: 0.4964Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
     3196/150528000 [..............................] - ETA: 2087:19:50 - loss: 19.826

     3241/150528000 [..............................] - ETA: 2087:14:06 - loss: 19.5706 - accuracy: 0.4955Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
     3243/150528000 [..............................] - ETA: 2087:13:16 - loss: 19.5595 - accuracy: 0.4955Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
     3246/150528000 [..............................] - ETA: 2087:13:19 - loss: 19.5425 - accuracy: 0.4957Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
     3249/150528000 [..............................] - ETA: 2087:12:55 - loss: 19.526

     3292/150528000 [..............................] - ETA: 2087:04:07 - loss: 19.2823 - accuracy: 0.4976Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
     3295/150528000 [..............................] - ETA: 2087:03:29 - loss: 19.2656 - accuracy: 0.4977Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
     3297/150528000 [..............................] - ETA: 2087:02:07 - loss: 19.2541 - accuracy: 0.4980Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
     3299/150528000 [..............................] - ETA: 2087:01:56 - loss: 19.242

Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
     3344/150528000 [..............................] - ETA: 2086:55:01 - loss: 19.0026 - accuracy: 0.4991Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
     3346/150528000 [..............................] - ETA: 2086:54:22 - loss: 18.9914 - accuracy: 0.4994Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
     3348/150528000 [..............................] - ETA: 2086:53:02 - loss: 18.9814 - accuracy: 0.4994Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2796 in device /job:localhost/replica:0/task:

In [ ]:
%tensorboard --logdir logs/vgg_minibatch

In [ ]:
print('train loss:', history.history['loss'][-1])
print('train accuracy:', history.history['accuracy'][-1])

print('dev loss:', history.history['val_loss'][-1])
print('dev accuracy:', history.history['val_accuracy'][-1])

fig, axs = plt.subplots(1,2,figsize=(15,5))

axs[0].plot(range(1,len(history.history['accuracy'])+1), history.history['accuracy'])
axs[0].plot(range(1,len(history.history['val_accuracy'])+1),history.history['val_accuracy'])
axs[0].set_title('Model Accuracy')
axs[0].set_ylabel('Accuracy')
axs[0].set_xlabel('Epoch')
axs[0].legend(['train', 'val'], loc='best')

axs[1].plot(range(1,len(history.history['loss'])+1),history.history['loss'])
axs[1].plot(range(1,len(history.history['val_loss'])+1),history.history['val_loss'])
axs[1].set_title('Model Loss')
axs[1].set_ylabel('Loss')
axs[1].set_xlabel('Epoch')
axs[1].legend(['train', 'val'], loc='best')

plt.show()

In [ ]:
# test_loss, test_acc = model.evaluate(x_test, y_test)

In [ ]:
# print('test loss:', test_loss)
# print('test accuracy:', test_acc)

In [ ]:
predictions = model.predict(x_test)

In [ ]:
print(predictions)

y_pred = []

for i in predictions:
    if i[0] < i[1]:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_mtx = confusion_matrix(y_test, y_pred)

In [ ]:
import seaborn as sns

f, ax = plt.subplots(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01, cmap="Greens",linecolor="gray", fmt='.1f', ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
from sklearn.metrics import classification_report

# Generate a classification report
report = classification_report(y_test, y_pred, target_names=['cat', 'dog'])

In [ ]:
print(report)

In [ ]:
print('Precision:- Accuracy of positive predictions.')
print('Precision = TP/(TP + FP)')
print('Recall:- Fraction of positives that were correctly identified.')
print('Recall = TP/(TP+FN)')
print('F1 score')
print('F1 Score = 2*(Recall * Precision) / (Recall + Precision)')
print('Accuracy : (TP+TN) / all')
print('macro avg = (normal+abnormal)/2 * precision or recall or f1 score')
print('weighted avg = normal/(normal+abnormal) * precision or recall or f1 score')